## Tratamento do arquivo 'evolucao.csv'
Neste caderno será apresentada a exploração inicial e tratamento dos dados do arquivo 'evolucao.csv'

### Importação das bibliotecas
 
A seguir uma breve apresentação de cada biblioteca que será utilizada:

- A biblioteca [NumPy](https://numpy.org/) é fundamental para qualquer tipo de computação científica em Python
- A biblioteca [pandas](https://pandas.pydata.org/) é a nossa ferramenta pricipal para análise e manipulação de dados

In [1]:
import numpy as np
import pandas as pd

### Leitura e tratamento inicial dos dados de entrada

Através da biblioteca `pandas` é realizada a leitura do arquivo '.csv' utilizando a função `read_csv()`, onde cada parâmetro tem o seguinte significado:

- `parse_dates=['Data']`: Fazer a importação da coluna `'Data'` como tipo `data hora`
- `encoding='iso-8859-1'`: Tipo de `encoding` do arquivo sendo lido
- `quotechar='"'`: O caracter usado para denotar o inicio e fim de um item entre aspas
- `delimiter='|'`: delimitador utilizado na escrita do arquivo

In [2]:
df = pd.read_csv('desafio-base1/evolucao.csv', parse_dates=['Data'], encoding='iso-8859-1', quotechar='"', delimiter='|')

(1230, 6)

### Visualização dos dados

Nas próximas duas células é realizada a visualização inicial dos dados.

Usando o método `head()` do `pandas` com um argumento `5` nele é possível visualizar os primeiros `5` registros do Dataframe.
    
O `.T` significa `Transposição`, desta forma as linhas serão visualizadas como colunas e vice-versa.

In [3]:
df.head(5).T

,0,1,2
Código,78,361,427
Data,2008-12-14 00:00:00,2009-03-27 00:00:00,2018-09-04 00:00:00
Procedimento,Consulta,Retorno,NaN
Tipo,HDA,HDA,Exame fisico
Medico,Teste 1,José,Teste 1
Evolução,Consequatur exercitationem adipisci. Labore qu...,Ducimus et error reprehenderit.\nAccusantium p...,Nesciunt quia ipsam quidem enim. Dolorum molli...


O método `info()` do `pandas` apresenta um resumo dos dados no Dataframe, uma informação interessante é o tipo de dado de cada recurso.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Código        1230 non-null   int64         
 1   Data          1230 non-null   datetime64[ns]
 2   Procedimento  1003 non-null   object        
 3   Tipo          1230 non-null   object        
 4   Medico        1230 non-null   object        
 5   Evolução      1230 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 57.8+ KB


## Limpeza e tratamento dos dados

A seguir será realizada a limpeza e tratamento dos dados.

### Valores ausentes, valores equivalentes e tratamento inicial

Quando utilizamos o método `info()` para ver o resumo dos dados, foi possível ver que muitas colunas tinham muitos dados ausentes, entrentanto, na documentação da iClinic é possível ver que os campos `patient_id`, `patient_name`, `physician_id`, `date` e `eventblock_text` são obrigatórios e no Dataset anterior as colunas `Código`, `Medico`, `Data` e `Evolução` são seus equivalentes diretos, porém, está faltando informações da coluna `patient_name`, pelo que será necessário um maior tratamento para cumprir as condições obrigatórias.

### Mudança de nome das colunas para o padrão iClinic

A seguir será realizada a mudança no nome dos recursos no Dataset anterior para os nomes equivalente ao padrão iClinic:

In [5]:
df = df.rename(
    columns = {
        "Código": "patient_id",
        "Data": "date",
        "Procedimento": "tab_name",
        "Tipo": "eventblock_name",
        "Evolução": "eventblock_text",
        "Medico": "physician_id",
    }
)

### Tratamento do recurso 'patient_name'

Este recurso precisa de informação externa para ser tratado, pelo que, após a importação do arquivo "patient.csv", onde está pode ser encontrada dentre outras informações, o nome do paciente, será utilizado o índice 'patient_id' para localizar o nome de cada usuário.

In [6]:
df_patient = pd.read_csv('desafio-base1-output/patient.csv', usecols = ["name"])

Repare que estamos utilizando o arquivo com os dados tratados anteriormente.
Neste caso, como o 'patient_id' inicia com 0 e está ordenada é fácil de encontrar seu nome correspondente.

In [7]:
index = df['patient_id'].values
df['patient_name'] = df_patient['name'][index].values

### Tratamento do recurso 'physician_id'

Este recurso precisa de informação externa para ser tratado, pelo que, após a importação do arquivo "physician_names.csv", poderá ser realizada a busca do nome do médico para logo substituir pelo seu Id correspondente:

In [8]:
df_physician_names = pd.read_csv('desafio-base1-output/physician_names.csv')

Repare que estamos utilizando o arquivo com os dados tratados anteriormente.
Neste caso, como o 'physician_names' está ordenado, podemos relacionar os indices de forma crescente e depois fazer a sua substituição devida. 

In [9]:
index_of_df_physician_names = pd.factorize(df_physician_names['physician_names'])[0].astype(int)
df['physician_id'] = df['physician_id'].replace(df_physician_names.values,index_of_df_physician_names)

### Adição de recursos ausentes

In [10]:
df['start_time'] = np.nan
df['end_time'] = np.nan

### Remoção de recursos repetidos ou não necessários

A seguir, serão removidos os recursos repetidos ou que não são mais necessários:

In [11]:
df = df.loc[:, ["patient_id","patient_name","physician_id","date","start_time","end_time","tab_name","eventblock_name","eventblock_text"]]

## Exportação do arquivo de saída

Como solicitado no desafio, o arquivo de saída será gerado com o conjunto de caracteres `UTF-8`:

In [12]:
df.to_csv('desafio-base1-output/event_record.csv',index=False, encoding='utf-8')